In [1]:
import numpy as np
import cv2
from numba import jit
from sympy import Line

COURT DETECTOR MODEL CLASS

In [51]:
class CourtDetector:

    """
    Detecting and tracking court in frame

    """
    def __init__(self):

        self.dist_tau = 3
        self.intensity_threshold = 40
        self.v_width = 0
        self.v_height = 0
        self.ymin = (2, 2)
        self.ymax = (267, 2)
        self.xmin = (2, 509)
        self.xmax = (267, 509)
        self.baseline_top = ((2, 2), (267, 2))
        self.baseline_top_inner = ((2, 37), (267, 37))
        self.baseline_bottom = ((2, 509), (267, 509))
        self.baseline_bottom_inner = ((2, 474), (267, 474))
        self.net = ((2, 255), (267, 255))
        self.left_court_line = ((2, 2), (2, 509))
        self.right_court_line = ((267, 2), (267, 509))
        self.left_inner_line = ((25, 2), (25, 509))
        self.right_inner_line = ((244, 2), (244, 509))
        self.middle_top = ((134, 2), (134, 184))
        self.middle_bottom = ((134, 327), (134, 509))
        self.top_inner_line = ((2, 184), (267, 184))
        self.bottom_inner_line = ((2, 327), (267, 327))

     
    def detect_court(self,image):

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #gray = cv2.equalizeHist(gray)
       #define white pixels threshold   
        gray1 = cv2.threshold(gray,170,255,cv2.THRESH_BINARY)[1]
        
       #filter white pixels into court line/not court line 
        filtered = self.filter_pixels(gray1)
       #get horizontal and vertical lines merged to get intersection points 
        horizontal, vertical = self.detect_lines(filtered,image)
        #self.display_lines_on_frame(image,horizontal,vertical)
        points = self.get_points(horizontal,vertical)

        edge = np.squeeze(cv2.convexHull(points))
        # limit contour to quadrilateral
        peri = cv2.arcLength(edge, True)
        corners = np.squeeze(cv2.approxPolyDP(edge, 0.01 * peri, True))
        corners = self.sort_intersection_points(corners)
        #for p in corners:
        #    cv2.circle(image, p, 1, (0, 0, 255), 10)
        Moments = cv2.moments(edge)

        if Moments["m00"] != 0:
            x = int(Moments["m10"] / Moments["m00"])
            y = int(Moments["m01"] / Moments["m00"])

            #return (edge,(x, y),points,corners)

        matrix, _ = cv2.findHomography(np.float32(np.array([self.ymin,self.ymax,self.xmin,self.xmax])), np.float32(corners), method=0)
        inv_matrix = cv2.invert(matrix)[1]

        self.find_lines_location(image,matrix)

        return image
        
 
    def filter_pixels(self,gray):
        
        """
        Filter pixels by using the court line structure
        """
        for i in range(self.dist_tau, len(gray) - self.dist_tau):
            for j in range(self.dist_tau, len(gray[0]) - self.dist_tau):
                if gray[i, j] == 0:
                    continue
                if (gray[i, j] - gray[i + self.dist_tau, j] > self.intensity_threshold and
                        gray[i, j] - gray[i - self.dist_tau, j] > self.intensity_threshold):
                    continue

                if (gray[i, j] - gray[i, j + self.dist_tau] > self.intensity_threshold and
                        gray[i, j] - gray[i, j - self.dist_tau] > self.intensity_threshold):
                    continue
                gray[i, j] = 0
        return gray

    def get_important_lines(self):
        """
        Returns all lines of the court
        """
        lines = [*self.baseline_top, *self.baseline_bottom, *self.baseline_top_inner, *self.baseline_bottom_inner, *self.net,
                 *self.left_court_line, *self.right_court_line,
                 *self.left_inner_line, *self.right_inner_line, *self.middle_top, *self.middle_bottom,
                 *self.top_inner_line, *self.bottom_inner_line]
        return lines

    def find_lines_location(self,frame,matrix):
        """
        Finds important lines location on frame
        """
        p = np.array(self.get_important_lines(), dtype=np.float32).reshape((-1, 1, 2))
        lines = cv2.perspectiveTransform(p, matrix).reshape(-1)
        p =  np.round(np.squeeze(cv2.perspectiveTransform(p,matrix))).astype(int)
        print(p)
        self.baseline_top = lines[:4]
        self.baseline_bottom = lines[4:8]
        self.baseline_top_inner = lines[8:12]
        self.baseline_bottom_inner = lines[12:16]
        self.net = lines[16:20]
        self.left_court_line = lines[20:24]
        self.right_court_line = lines[24:28]
        self.left_inner_line = lines[28:32]
        self.right_inner_line = lines[32:36]
        self.middle_top = lines[36:40]
        self.middle_bottom = lines[40:44]
        self.top_inner_line = lines[44:48]
        self.bottom_inner_line = lines[48:52]
        #if self.verbose:
        self.display_lines_on_frame(frame, [self.baseline_top, self.baseline_bottom, self.baseline_top_inner,
                                                       self.baseline_bottom_inner, self.net, self.top_inner_line, self.bottom_inner_line],
                                   [self.left_court_line, self.right_court_line,
                                    self.right_inner_line, self.left_inner_line, self.middle_top,  self.middle_bottom])


    def display_lines_on_frame(self, frame, horizontal=(), vertical=()):

        for line in horizontal:
            x1, y1, x2, y2 = line
            #print("horizontal : ",(int(round(x1)), int(round(y1))), (int(round(x2)), int(round(y2))))
            cv2.line(frame, (int(round(x1)), int(round(y1))), (int(round(x2)), int(round(y2))), (0, 255, 0), 2)
            cv2.circle(frame, (int(round(x1)), int(round(y1))), 1, (255, 0, 0), 2)
            cv2.circle(frame, (int(round(x2)), int(round(y2))), 1, (255, 0, 0), 2)

        for line in vertical:
            x1, y1, x2, y2 = line
            #print("vertical : ",(int(round(x1)), int(round(y1))), (int(round(x2)), int(round(y2))))
            cv2.line(frame, (int(round(x1)), int(round(y1))), (int(round(x2)), int(round(y2))), (0, 0, 255), 2)
            cv2.circle(frame, (int(round(x1)), int(round(y1))), 1, (255, 0, 0), 2)
            cv2.circle(frame, (int(round(x2)), int(round(y2))), 1, (255, 0, 0), 2)


    def classify_lines(self,lines):
        """
        Classify line to vertical and horizontal lines
        """
        horizontal = []
        vertical = []
        highest_vertical_y = np.inf
        lowest_vertical_y = 0
        for line in lines:
            x1, y1, x2, y2 = line
            dx = abs(x1 - x2)
            dy = abs(y1 - y2)
            if dx > 2 * dy:
                horizontal.append(line)
            else:
                vertical.append(line)
                highest_vertical_y = min(highest_vertical_y, y1, y2)
                lowest_vertical_y = max(lowest_vertical_y, y1, y2)

        # Filter horizontal lines using vertical lines lowest and highest point
        clean_horizontal = []
        h = lowest_vertical_y - highest_vertical_y
        lowest_vertical_y += h / 15
        highest_vertical_y -= h * 2 / 15
        for line in horizontal:
            x1, y1, x2, y2 = line
            if lowest_vertical_y > y1 > highest_vertical_y and lowest_vertical_y > y1 > highest_vertical_y:
                clean_horizontal.append(line)

        return clean_horizontal, vertical

    def line_intersection(self,line1, line2):

        l1 = Line(line1[0], line1[1])
        l2 = Line(line2[0], line2[1])

        intersection = l1.intersection(l2)

        return intersection[0].coordinates


    def merge_lines(self,horizontal_lines, vertical_lines):

        """
        Merge lines that belongs to the same frame`s lines
        """

        # Merge horizontal lines
        horizontal_lines = sorted(horizontal_lines, key=lambda item: item[0])
        mask = [True] * len(horizontal_lines)
        new_horizontal_lines = []
        for i, line in enumerate(horizontal_lines):
            if mask[i]:
                for j, s_line in enumerate(horizontal_lines[i + 1:]):
                    if mask[i + j + 1]:
                        x1, y1, x2, y2 = line
                        x3, y3, x4, y4 = s_line
                        dy = abs(y3 - y2)
                        if dy < 10:
                            points = sorted([(x1, y1), (x2, y2), (x3, y3), (x4, y4)], key=lambda x: x[0])
                            line = np.array([*points[0], *points[-1]])
                            mask[i + j + 1] = False
                new_horizontal_lines.append(line)

        # Merge vertical lines
        vertical_lines = sorted(vertical_lines, key=lambda item: item[1])
        xl, yl, xr, yr = (0, self.v_height * 6 / 7, self.v_width, self.v_height * 6 / 7)
        mask = [True] * len(vertical_lines)
        new_vertical_lines = []
        for i, line in enumerate(vertical_lines):
            if mask[i]:
                for j, s_line in enumerate(vertical_lines[i + 1:]):
                    if mask[i + j + 1]:
                        x1, y1, x2, y2 = line
                        x3, y3, x4, y4 = s_line
                        xi, yi = self.line_intersection(((x1, y1), (x2, y2)), ((xl, yl), (xr, yr)))
                        xj, yj = self.line_intersection(((x3, y3), (x4, y4)), ((xl, yl), (xr, yr)))

                        dx = abs(xi - xj)
                        if dx < 10:
                            points = sorted([(x1, y1), (x2, y2), (x3, y3), (x4, y4)], key=lambda x: x[1])
                            line = np.array([*points[0], *points[-1]])
                            mask[i + j + 1] = False

                new_vertical_lines.append(line)
        return new_horizontal_lines, new_vertical_lines

    def get_points(self,horizontal_lines, vertical_lines):

        Px=[]
        Py=[] 
        for lineh in horizontal_lines:
            xh1, yh1, xh2, yh2 = lineh
            for linev in vertical_lines:
                xv1, yv1, xv2, yv2 = linev
                x,y = self.line_intersection(((xh1, yh1), (xh2, yh2)), ((xv1, yv1), (xv2, yv2)))
                x,y = round(eval(str(x))),round(eval(str(y)))
                Px.append(x)
                Py.append(y)

        points = np.float32(np.column_stack((Px,Py)))

        return np.round(points).astype(int)


    def detect_lines(self,gray,frame):
        """
        Finds all line in frame using Hough transform
        """
        minLineLength = 100
        maxLineGap = 20
        self.v_height, self.v_width = frame.shape[:2]
        # Detect all lines
        lines = cv2.HoughLinesP(gray, 1, np.pi / 180, 100, minLineLength=minLineLength, maxLineGap=maxLineGap)
        lines = np.squeeze(lines)

        # Classify the lines using their slope
        horizontal, vertical = self.classify_lines(lines)

        # Merge lines that belong to the same line on frame
        horizontal, vertical = self.merge_lines(horizontal, vertical)

        return horizontal, vertical

    def sort_intersection_points(self,intersections):
        """
        sort intersection points from top left to bottom right
        """
        y_sorted = sorted(intersections, key=lambda x: x[1])
        p12 = y_sorted[:2]
        p34 = y_sorted[2:]
        p12 = sorted(p12, key=lambda x: x[0])
        p34 = sorted(p34, key=lambda x: x[0])
        p12 = list(map(list,p12[:]))
        p34 = list(map(list,p34[:]))

        return np.array(p12 + p34)

In [4]:
@jit
def test(model):

    cap = cv2.VideoCapture('D:/EFREI/project/Badminton_Project/test.mp4')
    #cap = cv2.VideoCapture('./data/dataset/local/videos/matches/3.mp4')
    #image = Image.open('./data/dataset/local/images/court/court.jpg')
    #court = np.array(image.resize((150,200)).convert('RGB'))
    #court = court[:, :, ::-1].copy()

    cap.set(1,0); 
    ret, img1 = cap.read()

    corners = model.detect_court(img1)

    #looping through all frames of the video
    while cap.isOpened():
        ret, frame = cap.read()

        #model.detect_court(frame)
       
        #cv2.imshow('Aerial View of the Court',court)
        #cv2.setWindowProperty('Aerial View of the Court', cv2.WND_PROP_TOPMOST, 1)
        for p in corners[3]:
            cv2.circle(frame, p, 1, (0, 255, 0), 10)

        frame = cv2.resize(frame, (frame.shape[1]// 2 ,frame.shape[0]// 2 ))
        cv2.imshow('Court Detection', frame)
    
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [11]:
model = CourtDetector()
test(model)

In [52]:
image = cv2.imread("D:/EFREI/project/Badminton_Project/data/images/8.PNG")
model = CourtDetector()
img1=model.detect_court(image)

[[ 608  395]
 [1121  399]
 [ 348  931]
 [1387  934]
 [ 599  414]
 [1130  418]
 [ 382  861]
 [1352  864]
 [ 522  571]
 [1209  576]
 [ 608  395]
 [ 348  931]
 [1121  399]
 [1387  934]
 [ 653  395]
 [ 439  931]
 [1077  399]
 [1298  934]
 [ 864  397]
 [ 865  513]
 [ 866  648]
 [ 868  933]
 [ 552  511]
 [1179  515]
 [ 486  646]
 [1246  650]]


In [53]:
#cv2.imshow('original',c)
img1 = cv2.resize(img1, (image.shape[1]// 2 ,image.shape[0]// 2 ))
cv2.imshow('mine',img1)
#cv2.imshow('not mine',img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
cv2.imwrite("./2.PNG",img1)

True